In [1]:
import os
code_path = os.path.dirname(os.path.abspath(".")).rsplit("/",1)[0]
import sys
sys.path.append(code_path + '/model_code')

import pandas as pd
import numpy as np
import torch
from dgl.dataloading import GraphDataLoader
import sys
import random
from Dataloader.dataloader import collate_fn, LeadOptDataset
from utilis.function import get_loss_func
from utilis.trick import Writer


In [2]:
code_path


'/home/yujie/code230118'

In [3]:
def setup_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def freezen(model):
    need_updata = ['FNN.0.weight', 'FNN.0.bias', 'FNN.2.weight', 'FNN.2.bias', 'FNN.4.weight', 'FNN.4.bias', 'FNN.6.weight', 'FNN.6.bias',
                   'FNN2.0.weight', 'FNN2.0.bias', 'FNN2.2.weight', 'FNN2.2.bias', 'FNN2.4.weight', 'FNN2.4.bias', 'FNN2.6.weight', 'FNN2.6.bias']

    for name, parameter in model.named_parameters():
        if name in need_updata:
            parameter.requires_grad = False
        else:
            parameter.requires_grad = True

In [4]:
def input_generation(file_name, ref, newly_mols):
    
    # /home/yujie/code230118/data/selection_graph/

    if os.path.exists(f"{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/") is False:
        os.system(f' mkdir "{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/" ')

    ligand1_tra = []
    ligand2_tra = []
    Lable1_tra = []
    Lable2_tra = []
    Lable_tra = []
    Rank1_tra = []
    Rank2_tra = []
    Rank_last_tra = []
    ligand1_num_tra = []
    ligand2_num_tra = []
    reference_num_tra = []

    ligand1_val = []
    ligand2_val = []
    Lable1_val = []
    Lable2_val = []
    Lable_val = []
    Rank1_val = []
    Rank2_val = []
    Rank_last_val = []
    ligand1_num_val = []
    ligand2_num_val = []
    reference_num_val = []


    # 加载该系列化合物中所有的化合物
    file_name_ = f"{code_path}/data/selection_graph/{file_name}/"

    ligands = [i for i in os.listdir(file_name_) if
               i.rsplit(".", 1)[-1] == "pkl" and i.rsplit(".", 1)[0] != "pocket"]

    ligands = [f"{code_path}/data/selection_graph/{file_name}/" + i for i in ligands]

    # 加载上一轮迭代之前所有的refs
    pd_for_refs = pd.read_csv(f"{code_path}/data/selection_graph/{file_name}/input_file/{ref-3}_reference/predict.csv")
    refs = list(set(pd_for_refs.Ligand2_num.values))
    # 和最新的3个ref合并
    refs.extend(newly_mols)
    refs = [f"{code_path}/data/selection_graph/{file_name}/{file_name}_" + i + ".pkl" for i in refs]

    # label
    label_csv = f"{code_path}/data/selection/{file_name}_pose/label.csv"
    df = pd.read_csv(label_csv)


    for i in range(len(refs)):

        reference_ligand = refs[i]

        reference_ligand_number = i + 1

        # finetune
        for query_ligand in refs:

            if query_ligand == reference_ligand:
                continue

            ligand1_val.append(query_ligand)
            ligand2_val.append(reference_ligand)

            ligand1_number = str(query_ligand.rsplit('.', 1)[0].split("/")[-1].split("_", 1)[1])
            ligand2_number = str(reference_ligand.rsplit('.', 1)[0].split("/")[-1].split("_", 1)[1])

            ligand1_num_val.append(ligand1_number)
            ligand2_num_val.append(ligand2_number)

            lable1 = float(df[df.name == ligand1_number].pIC50)
            lable2 = float(df[df.name == ligand2_number].pIC50)
            Lable1_val.append(lable1)
            Lable2_val.append(lable2)
            Lable_val.append(lable1 - lable2)

            index1 = ligands.index(query_ligand)
            index2 = ligands.index(reference_ligand)
            Rank1_val.append(index1)
            Rank2_val.append(index2)
            Rank_last_val.append(max(index1, index2))
            reference_num_val.append(reference_ligand_number)

        # predict
        for query_ligand in ligands:
            if query_ligand in refs:
                continue

            ligand1_tra.append(query_ligand)
            ligand2_tra.append(reference_ligand)

            ligand1_number = str(query_ligand.rsplit('.', 1)[0].split("/")[-1].split("_", 1)[1])
            ligand2_number = str(reference_ligand.rsplit('.', 1)[0].split("/")[-1].split("_", 1)[1])

            ligand1_num_tra.append(ligand1_number)
            ligand2_num_tra.append(ligand2_number)

            lable1 = float(df[df.name == ligand1_number].pIC50)
            lable2 = float(df[df.name == ligand2_number].pIC50)

            Lable1_tra.append(lable1)
            Lable2_tra.append(lable2)
            Lable_tra.append(lable1 - lable2)

            index1 = ligands.index(query_ligand)
            index2 = ligands.index(reference_ligand)
            Rank1_tra.append(index1)
            Rank2_tra.append(index2)
            Rank_last_tra.append(max(index1, index2))
            reference_num_tra.append(reference_ligand_number)

    df_predict = pd.DataFrame(
        {'reference_num': reference_num_tra,
         'Ligand1': ligand1_tra, "Ligand2": ligand2_tra, "Lable": Lable_tra, \
         "Lable1": Lable1_tra, "Lable2": Lable2_tra, \
         "Rank1": Rank1_tra, "Rank2": Rank2_tra, "Rank": Rank_last_tra, "Ligand1_num": ligand1_num_tra, \
         "Ligand2_num": ligand2_num_tra})

    df_finetune = pd.DataFrame(
        {'reference_num': reference_num_val,
         'Ligand1': ligand1_val, "Ligand2": ligand2_val, "Lable": Lable_val, \
         "Lable1": Lable1_val, "Lable2": Lable2_val, \
         "Rank1": Rank1_val, "Rank2": Rank2_val, "Rank": Rank_last_val, "Ligand1_num": ligand1_num_val, \
         "Ligand2_num": ligand2_num_val})

    new_ref_pre_csv =  f"{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/predict.csv"
    new_ref_fin_csv = f"{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/finetune.csv"

    df_predict.to_csv(new_ref_pre_csv, index=0)
    df_finetune.to_csv(new_ref_fin_csv, index=0)

In [5]:
def model_predict(file_name, ref, model, device, logger_writer,beita=-2):
    # 加载需要predict的数据
    test_dataset = LeadOptDataset(
        f"{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/predict.csv")
    test_dataloader = GraphDataLoader(test_dataset, collate_fn=collate_fn, batch_size=20,
                                      drop_last=False, shuffle=False)
    # 通过原始的文献获得predict的mol的信息
    t_pd = pd.read_csv(f"{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/predict.csv")
    names = t_pd.Ligand1_num.values

    # 预测
    pres = []
    labels = []
    model.eval()
    for batch_data in test_dataloader:
        graph1, graph2, pock, label, label1, label2, rank1, file_name = batch_data
        # to cuda
        graph1, graph2, pock, label, label1, label2 = graph1.to(device), graph2.to(device), pock.to(
            device), label.to(device), label1.to(
            device), label2.to(device)
        logits, _, _, _ = model(graph1,
                                graph2,
                                pock)
        pre = logits.squeeze() + label2
        labels += label1.tolist()
        pres += pre.tolist()

    result_df = pd.DataFrame({"pre": pres, "label": labels, "mol_name": names})

    if ref != 1:
        std_ = result_df.groupby('mol_name')[['pre', 'label']].std().reset_index().pre.values

    result_df = result_df.groupby('mol_name')[['pre', 'label']].mean().reset_index()
    if ref != 1:
        result_df["uncertainty"] = std_

    s = result_df[["pre", "label"]].corr(method='spearman').iloc[0, 1]
    p = result_df[["pre", "label"]].corr(method='pearson').iloc[0, 1]

    mol_name = result_df.mol_name.values
    pre_ic50 = result_df.pre.values
    if ref != 1:
        uncertainty = result_df.uncertainty.values
    if ref != 1:
        ucb = pre_ic50 + beita * uncertainty
    else:
        ucb = pre_ic50

    ind = np.argsort(ucb)[-3:][::-1]
    # ind = np.argsort(pre_ic50)[-3:][::-1]
    ind = mol_name[ind]

    logger_writer(f"Prediction:")
    logger_writer(f"    reference number: {ref}")
    logger_writer(f"    spearman: {s}")
    logger_writer(f"    pearson: {p}")
    logger_writer(f"    selected molecule: {ind}")
    logger_writer(f" ")

    return ind

In [6]:
def model_finetune(file_name, ref, model, device, logger_writer,lr=0.000001):
    train_dataset = LeadOptDataset(
        f"{code_path}/data/selection_graph/{file_name}/input_file/{ref}_reference/finetune.csv")
    train_dataloader = GraphDataLoader(train_dataset, collate_fn=collate_fn, batch_size=20,
                                       drop_last=False, shuffle=False)

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_func = get_loss_func("mse")
    for epoch in range(3):
        model.train()
        for batch_data in train_dataloader:
            graph1, graph2, pock, label, label1, label2, rank1, file_name = batch_data
            # to cuda
            graph1, graph2, pock, label, label1, label2 = graph1.to(device), graph2.to(device), pock.to(
                device), label.to(device), label1.to(
                device), label2.to(device)

            logits, logits_class, _, _ = model(graph1,
                                               graph2, pock)
            loss = loss_func(logits.squeeze(dim=-1).float(), label.float())

            opt.zero_grad()
            loss.backward()
            opt.step()

    logger_writer(f"Finetuned with {ref} molecules.")
    # print(f"Finetuned with {ref} molecules.")

    return model

In [7]:
best_mols = \
    {'FGFR2': ['lig_38'],
     'BCL6':  ['lig_1'],
     'HO1':   ['lig_7i'],
     'LRRK2': ['lig_11'],
     'sEH':   ['lig_11h'],
     'CDK9':  ['lig_16'],
     'WDR5':  ['lig_27'],
     'AAK1':  ['lig_(S)-32'],
     'PSK13': ['lig_37']}

device = "cuda:1"
seed = [1,2,3,4,5,6]

select_files = ["sEH", 'FGFR2', 'HO1', 'LRRK2', 'CDK9',  'WDR5', 'AAK1', 'BCL6', 'PSK13']

In [ ]:
Beita = [2,-2,0]

for beita in Beita:
    print(beita)
    for sd in seed:
        setup_seed(sd)
        print(f"seed:{sd}")
        for file_name in select_files:
            print(file_name)
            if file_name == 'CDK9':
                lr = 0.01
            # 记录
            logger_writer = Writer(f"{code_path}/record_activelearning.txt")
            logger_writer(file_name)

            model = torch.load(
                f"{code_path}/PBCNet.pth",
                map_location="cpu")
            model.to(device)
            # freezen(model)

            order = []
            for ref in range(1, 1000, 3):


                # 第一轮不需要finetune
                if ref == 1:
                    selected_mols = model_predict(file_name, ref, model, device, logger_writer,beita)
                else:
                    input_generation(file_name, ref, newly_mols=selected_mols)

                    # model = model_finetune(file_name, ref, model, device, logger_writer)
                    # selected_mols = model_predict(file_name, ref, model, device, logger_writer)

                    # connected
                    model2 = model_finetune(file_name, ref, model, device, logger_writer,lr)
                    selected_mols = model_predict(file_name, ref, model2, device, logger_writer,beita)

                # 判断新挑选的化合物小分子是否为最佳的(之一)
                n = 0
                for selected_mol in selected_mols:
                    n += 1
                    if selected_mol in best_mols[file_name]:
                        best_mols[file_name].remove(selected_mol)

                        order.append(ref - 1 + n)

                        logger_writer(f"{selected_mol} is selected!")
                        logger_writer(f"order: {ref - 1 + n}")
                        logger_writer(" ")

                # 判断是否需要继续迭代
                if not best_mols[file_name]:
                    logger_writer(f"The iteration of AL is over!")
                    break

            logger_writer(f"{file_name} each order is {order}, mean order is {np.mean(order)}")
            print(f"{file_name} each order is {order}, mean order is {np.mean(order)}")
            logger_writer(" ")
            logger_writer(" ")
            logger_writer(" ")